In [ ]:
# 首先安装/更新必要的库
!pip install -U bitsandbytes
!pip install -U transformers
!pip install flask gradio

# 导入必要的库
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gradio as gr
import torch

# 设置系统提示模板
prompt_style = """System: 您是一个专业的医生，专注于提供准确且简洁的回答，回答一定要简明扼要，并且提供具体的药物建议以及食用该药物的频率。
Question: {user_input}
Answer: """

# 创建4位量化配置
def load_model():
    print("正在加载模型，这可能需要几分钟...")
    try:
        # 设置4位量化参数
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )

        # 加载tokenizer和模型
        tokenizer = AutoTokenizer.from_pretrained("beita6969/DeepSeek-R1-Distill-Qwen-32B-Medical")
        model = AutoModelForCausalLM.from_pretrained(
            "beita6969/DeepSeek-R1-Distill-Qwen-32B-Medical",
            quantization_config=quantization_config,
            device_map="auto"
        )

        print("模型已成功加载")
        return tokenizer, model
    except Exception as e:
        print(f"模型加载失败: {e}")
        print("尝试使用备用模型...")

        # 尝试加载一个更小的备用医疗模型
        try:
            tokenizer = AutoTokenizer.from_pretrained("IDEA-CCNL/Ziya-LLaMA-13B-v1")
            model = AutoModelForCausalLM.from_pretrained(
                "IDEA-CCNL/Ziya-LLaMA-13B-v1",
                device_map="auto",
                load_in_8bit=True  # 使用8位量化以减少内存需求
            )
            print("备用模型加载成功")
            return tokenizer, model
        except Exception as e2:
            print(f"备用模型加载也失败了: {e2}")
            raise Exception("无法加载任何模型，请检查GPU内存或尝试使用更小的模型")

# 加载模型
tokenizer, model = load_model()

def format_answer(text):
    """美化答案格式，增加换行以提高可读性"""
    text = text.replace("。", "。\n")
    text = text.replace("？", "？\n")
    text = text.replace("！", "！\n")
    return text

def generate_answer(question):
    """生成医疗问题的回答"""
    if not question or question.strip() == "":
        return "请输入您的问题"

    user_input = question.strip()

    # 显示提示信息
    print(f"正在处理问题: {user_input}")

    # 准备输入
    inputs = tokenizer([prompt_style.format(user_input=user_input)], return_tensors="pt").to(model.device)

    # 生成回答
    with torch.no_grad():
        try:
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=256,
                no_repeat_ngram_size=3,
                temperature=0.1,
                repetition_penalty=1.2,
                do_sample=True
            )
        except RuntimeError as e:
            # 如果出现内存不足的错误，尝试减少batch_size或max_new_tokens
            if "CUDA out of memory" in str(e):
                print("内存不足，尝试降低生成参数...")
                outputs = model.generate(
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=128,  # 减少生成长度
                    no_repeat_ngram_size=3,
                    temperature=0.7,
                    repetition_penalty=1.2,
                    do_sample=False  # 关闭采样以减少内存使用
                )
            else:
                raise e

    # 解码并提取回答
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 分离出Answer部分
    if "Answer:" in response:
        answer = response.split("Answer:")[1].strip()
    else:
        answer = response.strip()

    # 格式化回答
    formatted_answer = format_answer(answer)
    return formatted_answer

# 定义Gradio界面
def create_ui():
    """创建Gradio用户界面"""
    with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as interface:
        gr.Markdown("# 🏥 医学问答助手")
        gr.Markdown("#### 基于DeepSeek-R1-Distill-Qwen-32B-Medical模型")

        with gr.Row():
            with gr.Column(scale=4):
                input_text = gr.Textbox(
                    label="请输入您的医学问题",
                    placeholder="例如：我最近感冒了，有什么药物可以缓解症状？",
                    lines=3
                )

                with gr.Row():
                    submit_btn = gr.Button("提交问题", variant="primary")
                    clear_btn = gr.Button("清除", variant="secondary")

            with gr.Column(scale=6):
                output_text = gr.Textbox(
                    label="医生回答",
                    lines=12,
                    show_copy_button=True
                )

        # 添加状态指示器
        status = gr.Textbox(label="状态", value="就绪", interactive=False)

        # 示例问题
        gr.Examples(
            examples=[
                "我最近总是头痛，特别是早上起床后，有什么药物可以缓解？",
                "高血压患者应该注意什么，有哪些药物可以控制血压？",
                "我的孩子发烧38.5度，该如何处理？需要服用什么药物？",
                "糖尿病患者的日常饮食应该注意什么？",
                "我经常失眠，有什么药物可以帮助改善睡眠质量？"
            ],
            inputs=input_text
        )

        # 设置事件处理
        def process_with_status(question):
            status.value = "正在生成回答..."
            try:
                answer = generate_answer(question)
                status.value = "就绪"
                return answer
            except Exception as e:
                error_msg = f"生成回答时出错: {str(e)}"
                status.value = "发生错误"
                return error_msg

        submit_btn.click(
            fn=process_with_status,
            inputs=input_text,
            outputs=output_text
        )

        input_text.submit(
            fn=process_with_status,
            inputs=input_text,
            outputs=output_text
        )

        clear_btn.click(fn=lambda: "", inputs=None, outputs=input_text)

        gr.Markdown("### 注意事项")
        gr.Markdown("* 本系统仅供参考，不能替代专业医生的诊断和建议")
        gr.Markdown("* 如有严重症状，请立即就医")
        gr.Markdown("* 药物使用请遵医嘱")

    return interface

# 创建并启动界面
demo = create_ui()

# 启动Gradio界面（在Colab中可以公开访问）
demo.launch(share=True, debug=True)

正在加载模型，这可能需要几分钟...


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:206: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

模型已成功加载
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f938b1faaeaec2b977.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在处理问题: 我的孩子发烧38.5度，该如何处理？需要服用什么药物？
正在处理问题: 我最近总是头痛，特别是早上起床后，有什么药物可以缓解？
